# <font color=#B2D732> <span style="background-color: #4424D6"> Spinal cord diffusion preprocessings </font>
<hr style="border:1px solid black">  

*Project: 2024_brsc_aging_project*  
*Paper: in prep*  
**@ author:** 
> Caroline Landelle, caroline.landelle@mcgill.ca // landelle.caroline@gmail.com   
> June 2024 (last update: 11 April 2025)

**Description:** 
> This notebook provides code for diffusion MRI data of spinal cord acquisition.
> The second line of the section "imports & configuration" of this notebook should be modify with the good path 


**Associated files:**
> - ../../config/diffusion/01_sc_preprocess_diffusion.json ; This file contains the directories and should be modified first
> - ../../config/participants_brsc_*.tsv ; this file contains the information about the participants and should be updated after each new acquisition
> - ../brsc_preprocess.py ; Nothing should be changed in this file
> - ../BrSc_utils.py; Nothing should be changed in this file
> - You need to copy the version sct6.1 of the template and atlas that include spinal segmental levels + WM tracts and put the path in the config file "PAM50_sct_dir":".../template_sct6.1",
 

**Steps:**
> **I. DWI image preparation**    
> **II. Segmentation**    
> **III. Motion correction**   
<hr style="border:1px solid black">



## <font color=#dd9c07>  Imports & configuration </font >

In [1]:
import sys, json, glob, os, shutil
code_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project/"
os.chdir(code_dir)
sys.path.append(code_dir + "/code/") # Change this line according to your directory

from brsc_preprocess import Preprocess_Sc
from brsc_preprocess import Preprocess_DWI_Sc


#
%load_ext autoreload
%autoreload 2
%matplotlib inline
    
# Load config file ------------------------------------------------------------
# The config file should be first review and all path should be modified (lines: )
with open(code_dir + '/config/preprocessing/03_sc_preprocess_microstructural.json') as config_file: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_file) # load config file should be open first and the path inside modified

config["participants_IDs"]=config["participants_IDs_DWI"]
dwi=Preprocess_DWI_Sc(config) # initialize the function
preprocess_Sc=Preprocess_Sc(config) # initialize the function


/cerebro/cerebro1/dataset//stratals/rawdata//sub-A006_test//dwi/sub-A006_test_dwi.nii.gz


<hr style="border:1px solid black">

### <font color=#dd9c07> I. Separate B0 and DWI volumes </font>

In [2]:
dwi_files=dwi.separate_b0_dwi(redo=False,verbose=True)

DWI and b0 images separation was already done for sub-A006_test


### <font color=#dd9c07> II. Motion correction </font>

##### Segment the mean dwi image to initiate the coarse image for moco

In [3]:
seg_files=dwi.segmentation(i_files=dwi_files["dwi_mean"],redo=False,verbose=True)

DWI segmentation was already done for sub-A006_test


##### Run moco

In [4]:
moco_files=dwi.moco(seg_files=seg_files,redo=False,verbose=True)

DWI motion correction was already done for sub-A006_test


### <font color=#dd9c07> III. Segmentation </font>

In [5]:
seg_moco_files=dwi.segmentation(i_files=moco_files["moco_mean"],redo=False,verbose=True)

DWI segmentation was already done for sub-A006_test


### <font color=#dd9c07> V. Compute diffusion tensor imaging </font>

In [6]:
dti_files=dwi.compute_dti(i_files=moco_files["moco"],redo=False,verbose=True)

DWI diffusion tensor computation was already done for sub-A006_test


### <font color=#dd9c07> IV. Coregistration to PAM50 templae </font>

In [7]:
coreg_dir=[];dwi2PAM50=[];dwi2PAM50_dirs=[];o_warp_imgs=[];o_warpinv_imgs=[]
warpT1w_PAM50_files=[]; warpPAM50_T1w_files=[]
for ID_nb, ID in enumerate(config["participants_IDs"]):
    
    # output directory
    coreg_dir.append(os.path.dirname(dwi_files['dwi'][ID_nb]) + "/coregistration/")
    if not os.path.exists(coreg_dir[ID_nb]):
        os.mkdir(coreg_dir[ID_nb])

    # Load T1toPAM50warping field
    preprocess_dir=config["main_dir"]+ config["preprocess_dir"]["bmpd_dir"] if ID[0]=="P" else config["main_dir"] +config["preprocess_dir"]["main_dir"]
    
    warpT1w_PAM50_files.append(glob.glob(preprocess_dir + "sub-" + ID + "/anat/"+ config["preprocess_dir"]["T1w_sc_coreg"] + "*from-T1w_to-PAM50*")[0])
    warpPAM50_T1w_files.append(glob.glob(preprocess_dir + "sub-" + ID + "/anat/"+ config["preprocess_dir"]["T1w_sc_coreg"] + "*from-PAM50_to-T1w*")[0])

    # Coregister moco
    dwi2PAM50.append(preprocess_Sc.coreg_img2PAM50(ID=ID,
                                            i_img=moco_files["moco_mean"][ID_nb],
                                            i_seg=seg_moco_files[ID_nb],
                                            PAM50_cord=config["tools_dir"]["main_codes"] +config["PAM50_cord"],
                                            PAM50_t2=config["tools_dir"]["main_codes"] +config["PAM50_t1"],#coreg_type="centermass",
                                            ses_name=config["design_exp"]["ses_names"][0],
                                            o_folder=coreg_dir[ID_nb],
                                            initwarp=warpT1w_PAM50_files[ID_nb],initwarpinv=warpPAM50_T1w_files[ID_nb],
                                            img_type="dwi",
                                            redo=False,verbose=True))


    # wrap PAM50 files into DWI space, this step requiered the v6.1 of the template to include the segmental levels (config["PAM50_sct_dir"])
    if not os.path.exists(dwi2PAM50[ID_nb][0] + "/template/PAM50_cord.nii.gz"):
        string=f"sct_warp_template -d {moco_files['moco_mean'][ID_nb]} -w {dwi2PAM50[ID_nb][2]} -a 1 -ofolder {dwi2PAM50[ID_nb][0]} -t {config['PAM50_sct_dir']}"
        os.system(string)
        # add home made atlas (spinal levels from icaps)) 
        function="/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.6.0/bin/isct_antsApplyTransforms"
        for atlas in ["_" , "_gm","_wm"]:
            string=function + " -d 3 -i " +config["PAM50_sct_dir"] + "/template/PAM50_Frostell"+atlas+"28Parcels_7Networks.nii.gz -o " +dwi2PAM50[ID_nb][0] +"/template/PAM50_Frostell"+atlas+"28Parcels_7Networks.nii.gz -t " + dwi2PAM50[ID_nb][2] +" -r " +moco_files['moco_mean'][ID_nb] + " -n NearestNeighbor"
            
        
        for wm_atlas in ["3tracts"]:
            string=function + " -d 3 -i " +config["PAM50_sct_dir"] + "/template/PAM50_wm_atlas_"+wm_atlas+".nii.gz -o " +dwi2PAM50[ID_nb][0] +"/template/PAM50_wm_atlas_"+wm_atlas+".nii.gz -t " + dwi2PAM50[ID_nb][2] +" -r " +moco_files['moco_mean'][ID_nb]  + " -n NearestNeighbor"
        
        os.system(string)

    
    # Coregister DTI files
    for metric in ["FA","MD","RD","AD"]:
        preprocess_Sc.apply_warp(i_img=dti_files[metric][ID_nb],
                                 ID=ID,
                   o_folder=os.path.dirname(dti_files[metric][ID_nb]) +"/",
                   dest_img=config["tools_dir"]["main_codes"] + config["PAM50_t1"],
                   warping_field=warpT1w_PAM50_files[ID_nb],
                   tag='_inPAM50',redo=False,verbose=True,n_jobs=1)
    

>>>>> Registration between func image and PAM50 already exists for sub-A006_test
 
Tranformation was already applied put redo=True to redo that step
Tranformation was already applied put redo=True to redo that step
Tranformation was already applied put redo=True to redo that step
Tranformation was already applied put redo=True to redo that step
